In [1]:
import pandas as pd
import numpy as np

In [29]:
# Level 0 
level_0 = ["lokasi", "biaya", "fasilitas"]

# Level 1
level_1_lokasi = ["<3k", "4-5km", ">5km"]
level_1_biaya = [">2jt", "3-4jt", "<5jt"]
level_1_fasilitas = ["krg lengkap", "lengkap", "sgt lengkap"]

# Level 2 (Alternatives)
alternatives = ["sma1", "sma2", "sma3"]

In [21]:
# Input perbandingan kriteria (pairwise comparison)
def input_perbandingan_kriteria(list_criteria):
    df = pd.DataFrame(index=list_criteria, columns=list_criteria)
    for i in range(len(list_criteria)):
        for j in range(i+1, len(list_criteria)): 
            print(list_criteria[i], "vs", list_criteria[j])
            v = float(input("Masukkan nilai..."))

            df.loc[list_criteria[i], list_criteria[j]] = 1/v
            df.loc[list_criteria[j], list_criteria[i]] = v
    else:
        np.fill_diagonal(df.values, 1)

    return df

In [8]:
df_level_0 = input_perbandingan_kriteria(level_0)

lokasi vs biaya
lokasi vs fasilitas
biaya vs fasilitas


In [18]:
df_level_1_lokasi = input_perbandingan_kriteria(level_1_lokasi)

<3k vs 4-5km
<3k vs >5km
4-5km vs >5km


In [22]:
df_level_1_biaya = input_perbandingan_kriteria(level_1_biaya)

>2jt vs 3-4jt
>2jt vs <5jt
3-4jt vs <5jt


In [23]:
df_level_1_fasilitas = input_perbandingan_kriteria(level_1_fasilitas)

krg lengkap vs lengkap
krg lengkap vs sgt lengkap
lengkap vs sgt lengkap


In [13]:
ri_dict = {
    "1": 0.00,
    "2": 0.00,
    "3": 0.58,
    "4": 0.90,
    "5": 1.12,
    "6": 1.24,
    "7": 1.32,
    "8": 1.41,
    "9": 1.45,
    "10": 1.49
}

def process(df_pairwise):
    criteria = df_pairwise.columns

    # hitung total vertikal
    list_sum = []
    for i in range(len(df_pairwise.columns)):
        list_sum.append(df_pairwise.iloc[:, i].sum())
    temp = pd.DataFrame(index=["total"], columns=df_pairwise.columns)
    temp.loc["total"] = list_sum
    df_pairwise = pd.concat([df_pairwise, temp])
    total_awal = df_pairwise.loc["total"].copy()

    # Normalisasi
    for i in range(len(df_pairwise.columns)):
        total = df_pairwise.iloc[-1, i]
        for j in range(len(df_pairwise.columns)):
            df_pairwise.iloc[j, i] = df_pairwise.iloc[j, i] / total    

    # Hitung jumlah vertikal (lagi)
    list_sum = []
    for i in range(len(df_pairwise.columns)):
        list_sum.append(df_pairwise.iloc[:-1, i].sum())
    df_pairwise.loc["total"] = list_sum

    # Hitung jumlah horizontal
    list_sum = []
    for i in range(len(df_pairwise.index)):
        list_sum.append(df_pairwise.iloc[i, :].sum())

    df_pairwise['jumlah'] = list_sum

    # Prioritas
    df_pairwise["prioritas"] = df_pairwise["jumlah"] / len(criteria)

    # Eigen Values
    list_prioritas = df_pairwise["prioritas"].iloc[:-1].to_list()
    list_total_awal = total_awal.to_list()
    eigen_values = [list_prioritas[i] * list_total_awal[i] for i in range(len(list_prioritas))]
    eigen_values.append(sum(eigen_values))
    df_pairwise["eigen_value"] = eigen_values

    # Hitung CI
    total_eigen = df_pairwise["eigen_value"].iloc[-1]
    ci = (total_eigen-len(criteria)) / (len(criteria)-1)

    # Hitung CR
    ri = ri_dict[str(len(criteria))]
    cr = ci/ri

    return df_pairwise, ci, ri, cr

In [24]:
df_pairwise_0, ci_0, ri_0, cr_0 = process(df_level_0)
df_pairwise_lokasi, ci_lokasi, ri_lokasi, cr_lokasi = process(df_level_1_lokasi)
df_pairwise_biaya, ci_biaya, ri_biaya, cr_biaya = process(df_level_1_biaya)
df_pairwise_fasilitas, ci_fasilitas, ri_fasilitas, cr_fasilitas = process(df_level_1_fasilitas)

In [34]:
df_alternatives = pd.DataFrame(index=alternatives, columns=level_0)
data = dict(
    sma1 = [">5km", "3-4jt", "lengkap"],
    sma2 = ["4-5km", ">2jt", "krg lengkap"],
    sma3 = ["<3k", "<5jt", "sgt lengkap"]
)

for k in data:
    df_alternatives.loc[k, :] = data[k]

df_alternatives

,lokasi,biaya,fasilitas
sma1,>5km,3-4jt,lengkap
sma2,4-5km,>2jt,krg lengkap
sma3,<3k,<5jt,sgt lengkap


In [35]:
df_pairwise_0

,lokasi,biaya,fasilitas,jumlah,prioritas,eigen_value
lokasi,0.111111,0.076923,0.130435,0.318469,0.106156,0.955407
biaya,0.333333,0.230769,0.217391,0.781494,0.260498,1.128824
fasilitas,0.555556,0.692308,0.652174,1.900037,0.633346,0.971130
total,1.0,1.0,1.0,3.000000,1.000000,3.055361


In [58]:
prio_level_0 = df_pairwise_0["prioritas"].iloc[:-1].to_dict()
prio_lokasi = df_pairwise_lokasi["prioritas"].iloc[:-1].to_dict()
prio_biaya = df_pairwise_biaya["prioritas"].iloc[:-1].to_dict()
prio_fasilitas = df_pairwise_fasilitas["prioritas"].iloc[:-1].to_dict()

In [50]:
df_alternatives

,lokasi,biaya,fasilitas
sma1,>5km,3-4jt,lengkap
sma2,4-5km,>2jt,krg lengkap
sma3,<3k,<5jt,sgt lengkap


In [51]:
hasil_akhir = pd.DataFrame(index=alternatives, columns=level_0)
hasil_akhir

,lokasi,biaya,fasilitas
sma1,NaN,NaN,NaN
sma2,NaN,NaN,NaN
sma3,NaN,NaN,NaN


In [68]:
for alt in df_alternatives.index:
    for crit in df_alternatives.columns:
        # Nilai alternatif
        n_a = df_alternatives.loc[alt, crit]
        # n_a0 = nilai prioritas level 0
        n_a0 = prio_level_0[crit]
        
        if crit == "lokasi":
            # n_h = nilai akhir
            n_h = n_a0 * prio_lokasi[n_a]
        elif crit == "biaya":
            n_h = n_a0 * prio_biaya[n_a]
        else:
            n_h = n_a0 * prio_fasilitas[n_a]

        hasil_akhir.loc[alt, crit] = n_h

In [70]:
# hitung total horizontal
list_sum = []
for i in range(len(hasil_akhir.index)):
    list_sum.append(hasil_akhir.iloc[i, :].sum())

hasil_akhir['jumlah'] = list_sum

In [72]:
hasil_akhir

,lokasi,biaya,fasilitas,jumlah
sma1,0.066529,0.074334,0.135053,0.275917
sma2,0.027957,0.056792,0.05404,0.138789
sma3,0.011671,0.129372,0.444252,0.585295
